In [1]:
from FAdo.reex import *

In [2]:
"""
atom -> val
star -> arg
other -> arg1 /\ arg2
"""
def eTree(reg_exp,path):
    node = Node(reg_exp,path)
    if type(reg_exp)==position:
        return
    elif type(reg_exp)==star:
        son = eTree(node.exp.arg,path+"1")
    else:
        son1 = eTree(node.exp.arg1,path+"1")
        son2 = eTree(node.exp.arg2,path+"2")

In [3]:
NT = {}
class Node:
    
    def __init__(self,expr,string):
        self.type = type(expr)
        self.path = string
        self.exp = expr
        if self.type == position:
            self.posix = expr.val[1]
        NT[self.path] = self
    
    def set_left(self,n):
        NT[self.path+'1'] = n
        
    def set_right(self,n):
        NT[self.path+'2'] = n

    def get_left(self):
        return NT.get(self.path+'1')
    
    def get_right(self):
        return NT.get(self.path+'2')
        
    def get_parent(self):
        if self.path=="1":
            return NT[self.path]
        return NT[self.path[:-1]]
    
    def printTree(self):
        if NT.get(self.path):
            print self.exp
            if NT.get(self.path+'1'):
                l = NT[self.path+'1']
                l.printTree()
            if NT.get(self.path+'2'):
                r = NT[self.path+'2']
                r.printTree()
        
    def get_root(self):
        return NT.get("1")
    
    def get_brother(self):
        if self.path[len(self.path)-1]==1:
            node = self.get_parent().get_right()
        else:
            node = self.get_parent().get_left()
        return node
    
    def check_reex(self):
        if self.type == 'atom':
            if self.get_left() or self.get_right():
                return False
        elif self.type == 'star':
            if self.get_left() is None or self.get_right():
                return False
            if self.get_left().check_reex() is False:
                return False
        else:
            if self.get_left() is None or self.get_right() is None:
                return False
            if self.get_left().check_reex() is False or self.get_right().check_reex() is False:
                return False
        return True
    
    def reflexive(self,n):
        if n == None:
            return False
        if LCA(self,n)==n.path:
            return True
        else:
            return False

    def followList(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if follow(self,node):
                f_list.append(node.exp)
        return f_list
    
    # list of first atoms from a reg exp using lemma 2.3
    def first(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if isFirst(self,node):
                f_list.append(node.exp)
        return f_list
    
    # list of last atoms from a reg exp using lemma 2.3
    def last(self):
        f_list = []
        for x in NT:
            node = NT.get(x)
            if isLast(self,node):
                f_list.append(node.exp)
        return f_list
    

def get_root():
    return NT.get("1")

# Last Common Ancestor
def LCA(p,q):
    str1 = p.path
    str2 = q.path
    ret = ""
    i = 0
    while i < len(str1) and i < len(str2):
        if str1[i]!=str2[i]:
            break
        ret = ret+str1[i]
        i = i+1
    return ret

# Lowest Star Ancestor
def LSA(n):
    while n.type != star:
        if n.path == "1":
            return None
        n = n.get_parent()
    return n

In [4]:
def followList():
    l = {}
    atoms = []
    for x in NT:
        if NT.get(x).type == position:
            atoms.append(NT.get(x))
            l[NT.get(x).exp]=[]
    for x in atoms:
        f_list = []
        for w in atoms:
            if follow(x,w):
                f_list.append(w.exp)
        l[x.exp]=f_list
    return l

In [5]:
def follow(p,q):
    node1 = NT.get(LCA(p,q))
    if node1.type==concat and q.exp in node1.get_right().exp.first() and p.exp in node1.get_left().exp.last():
        return True
    node1 = LSA(node1)
    if node1 == None:
        return False
    if q.exp in node1.exp.first() and p.exp in node1.exp.last():
        return True
    return False

In [6]:
# boolean that states if an atom is the first from the right child of a concatenation
def SupFirst(n):
    father = n.get_parent()
    if father.type == concat and father.get_right()==n and not father.get_left().exp.ewp():
        return True
    return False

# boolean that states if an atom is the last from the left child of a concatenation
def SupLast(n):
    father = n.get_parent()
    if father.type == concat and father.get_left()==n and not father.get_right().exp.ewp():
        return True
    return False

# pointer to the first(right child) of a concatenation
def pSupFirst(n):
    while not SupFirst(n):
        if n.path == "1":
            return None
        n = n.get_parent()
    return n

# pointer to the last(left child) of a concatenation
def pSupLast(n):
    while not SupLast(n):
        if n.path == "1":
            return None
        n = n.get_parent()
    return n
        

In [7]:
# lemma 2.3
def isFirst(n,p):
    if p.type != position or n.type == position:
        return False
    k = pSupFirst(p)
    if k != None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    return False

def isLast(n,p):
    if p.type != position or n.type == position:
        return False
    k = pSupLast(p)
    if k != None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    return False

In [8]:
# epsilon acceptance by lemma 2.6
def ewp_SupFirst(p,q):
    node = pSupFirst(q)
    if node.path == "1" or p.type != position or q.type != position or not follow(p,q):
        return False
    node = node.get_parent()
    if node.reflexive(pSupLast(p)):
        return True
    return False